# An exercise in basic data science skills

## Summary

This notebook was developped in order to practice skills I have learned following the first part of the DataQuest data science tutorial. But, it can be of use to anyone who is interested in learning the basics of python since it uses the jupyter book format which is ideal for learning or trying coding (or so people say). 

The fictional goal of the present exercise is to help a company that builds Android and iOS mobile apps. Since most of their profits come from advertisements on said applications (they only produce free apps), the goal is to find what types of free apps are most likely to be downloaded. Note: the market is english speaking users. 

The exercise consists of 4 analysis steps:
 1. Downloading the file and separating the headers.
 2. Visually explore the data.
 3. Cleaning the data.
 4. Analysis proper in order to provide insights to the fictional company. 



This exercise uses the following two datasets:
https://www.kaggle.com/lava18/google-play-store-apps/home
https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps/home

    Note: I have inserted notes such as this one throught the comments. Most of them simply state which lines of codes MUST be run in order for the whole code to function. I also added some notes concerning ways to improve the code. The reason these improvements were not added was for the results of this notebook to be comparable to those of the tutorial. 

## Step 1
**The first step is to open the files and separating them from the headers (name of the columns)**
. Although the separation of the header from the rest of the file is not mandatory, it will make reading the rest of the information easier. 

     Note: running the cell below is MANDATORY for the rest of the code to work. 

In [ ]:
#Opening the google data set
from csv import reader
opfil = open('googleplaystore.csv', encoding='utf8')
tempgoogfil = list(reader(opfil))

hdrgoog = tempgoogfil[0]           #header
filgoog = tempgoogfil[1:]          #body

#Opening the apple data set
opfil = open('AppleStore.csv', encoding='utf8')
tempappfil = list(reader(opfil))

hdrapp = tempappfil[0]           #header
filapp = tempappfil[1:]          #body

## Step 2
**The second step is to explore the data and get an idea of what it contains**
 
 In the second step we specify a function that will be used to show the data (the function was kindly provided by dataquest).

    Note: running the cell below is MANDATORY for the rest of the code to work. 

In [ ]:
def explore(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Now the data sets can be explored. 

    Note: running the cell below is FACULTATIVE as it is simply meant to explore the data. 

In [ ]:
print(hdrgoog)
explore(filgoog,0,1)

print('\n') #creates new line to space information

print(hdrapp)
explore(filapp,0,1)

**A short legend for each of the datasets is now in order.**

The legends were taken from the website where the apps are downloaded

**Google dataset**

    App............... Name
    Category.......... Type of app
    Rating............ Average rating of the app
    Reviews........... # of revies
    Size.............. size of app
    Installs.......... Number of times the app was downloaded
    Type.............. Paid or free
    Price.............
    Content Rating.... Age group it is allowed for
    Genres............ Tool, entertainement, etc
    Last updated......
    Current version...
    Android version...
  
**Apple dataset**

    id................ Apple id
    track_name........ App name
    size.............. in bytes
    currenty        
    price           
    rating_count_tot.. number or ratings
    rating_cout_ver... number of ratings for current version
    user_rating....... average rating
    user_rating_ver... average rating of current version
    ver............... latest version
    cont_rating....... age group allowed for
    prime_genre....... game, weather, etc
    sup_devices.num... number of downloads
    ipadSc_urls.num... Number of screenshot for the display
    lang.num.......... number of supported languages
    vpp_lic........... vpp device based licensing enabled

## Step 3 
**This step involves cleaning the data (removing errors and unwanted data).**

### 3.1 Badly formated data
Interestingly, a quick visit on the discussion forum for the data shows that there is an error in one of the data entries. Something about the number of columns not being right. To verify whether it has been corrected and whether other errors existed, I wrote the following function. It searches through every row of the data to make sure that they possess the right number of columns. 

    Note: running the next 2 cells below is MANDATORY for the rest of the code to work. 

In [ ]:
def detect_wrong_col(data,ncol):
    count = 0
    for rr in data:
        if len(rr) != ncol:
            print(count)
            print(rr)
            print('\n')            
        count += 1

In [ ]:
print('Android')
detect_wrong_col(filgoog,len(hdrgoog))
print('Apple')
detect_wrong_col(filapp,len(hdrapp))

**!** We can see after running the function that there is inndeed still an error in the google data set. That is, there is a column missing. One way to correct this problem is to delete the entry.  

    Note: running the cell below is MANDATORY for the rest of the code to work. But it must be ran only once. It should also be specified that a more ideal step would have been to create a new variable that contained the cleaned up data rather than deleting it. 

In [6]:
del(filgoog[10472]) #Running this line removes the duplicate elements (beware running it more than once, for it will remove good data)

### 3.2 Duplicate data
Another issue reported on the discussion forums for the data set was that there were duplicate elements. That is, some apps were present more than once. The reason why applications are reported more than once is unknown. To find out, I wrote the following function which will go through the data and identify which apps have duplicate entries and which don't. It also gives an output that can be used to visualize how duplicate entries differ between each other in order to better understand the source of duplicates. 

    Note: running the following two cells is FACULTATIVE for the rest of the code. Their main use is to inform the user as to the nature of the data. Additionally, the function was improved from that suggested by the turotial The current one has more outputs that could allow a more in depth search of the duplicate entries. 

In [7]:
def find_dupl(data,hdr,nam_col_idx):
    #this script will search for duplicate data
    nci = nam_col_idx
    count = 0
    ulist = [] 
    tempdlist = []

    
    #first loop to obtain list of the names
    for rr in data:
        nam = rr[nci]
        if nam in ulist:
            tempdlist.append(nam)
        else:
            ulist.append(nam)
         
    dlist = set(tempdlist) #list ofunique apps without duplicates
    
    #second loop to compare data for each duplicate app
    ncol = len(data[1])
    idxdif = []                       #col which show differences across duplicates  
    detailed_output = []
    for nn in dlist:
        first = []                    #used to check whether this is the first entry of an app that later has duplicates
        count = 0
        for rr in data:
            if rr[nci] == nn:         #find instance of the same app
                if first:             #check whether it is the first time that this app is encountered
                    for co in range(0,ncol-1):    #loops through each colum and compareds them with first instance of app to see where they differ
                        if first[co] != rr[co]:
                            detailed_output.append([count,hdr[co],first[co],rr[co]])
                            
                            idxdif.append(co)
                else:
                    first = rr
                    
            count += 1
            
            
            
    return [idxdif,ulist,dlist,detailed_output]
            
            


Then when applied to files we get the following results:


In [8]:
idxg,uniquesg,duplicatesg,det_outg = find_dupl(filgoog,hdrgoog,0)
print(set(idxg))
#explore(det_outg,0,20)
for ii in range(0,20):
    print(det_outg[ii])
    

print('\n\n')

print(set(idxg))
idxa,uniquesa,duplicatesa,det_outa = find_dupl(filapp,hdrapp,2)
#explore(det_outa,0,20)
for ii in range(0,20):
    print(det_outa[ii])

{1, 2, 3, 4, 5, 6, 7, 9, 10, 11}
[5294, 'Reviews', '1343866', '1343106']
[2489, 'Reviews', '53743', '53747']
[3943, 'Reviews', '78158306', '78128208']
[3215, 'Reviews', '359403', '359560']
[7937, 'Reviews', '10979062', '10981850']
[293, 'Reviews', '1002861', '1002859']
[8747, 'Reviews', '21314', '21381']
[4068, 'Reviews', '2788923', '2788460']
[1735, 'Reviews', '5234162', '5234825']
[1757, 'Reviews', '5234162', '5234810']
[1891, 'Reviews', '5234162', '5235294']
[1921, 'Reviews', '5234162', '5235294']
[4227, 'Reviews', '5234162', '5231553']
[1855, 'Reviews', '214777', '214819']
[5415, 'Reviews', '666521', '666246']
[2225, 'Reviews', '1574197', '1574204']
[9680, 'Reviews', '1574197', '1574546']
[3976, 'Reviews', '815893', '815280']
[4989, 'Reviews', '85858', '85782']
[8450, 'Reviews', '552441', '552635']



{1, 2, 3, 4, 5, 6, 7, 9, 10, 11}
[5603, '', '4000', '7579']
[5603, 'id', '952877179', '1089824278']
[5603, 'size_bytes', '169523200', '240964608']
[5603, 'rating_count_tot', '107', '6

**Comment:** Notice that not all outputs of the functions were used. Originally, the different outputs where meant to explore different elements of the data. However, it was particularly obvious from the results displayed above that differences between duplicates were not limited to a specific column but were found across columns. However, the android datasets clearly shows that most differences are found for the number of reviews. This suggests that data were acquired a different times with some having been reviewed by more users. 

    Note: no duplicates were removed from the Apple Store dataset to keep my results coherent with theirs. Obviously, this should not be avoided in an analysis for a real company. 
    

Thus, when removing the duplicate data, we should aim to keep the most recent entry for each app. This should be the one that contains the highest number of reviews. We will do so using a two step procedure. 
  - In the first step, we link every unique app to the highest number of review reported for it. To do this, we create a dictionary.
  - In the second step, we use the dictionary to clean up the data by selecting only rows where the app's number of review is the highest.

Both steps are implemented by the function I wrote below. 

    Note: running the following two cells is MANDATORY for the rest of the code to funciton properly. 

In [9]:
def clean_data(data,col_nam,col_decision,decision='Highest'):
    #This function is meant to go through data and create a new dataset where duplicate entries are deleted and only
    #the one meeting a specific criteria is kept. The column for the criteria must be specified as well as the criteria.
    #However, right now, only 'highest' is implemented
    
    coln = col_nam
    cold = col_decision
    
    #creating dictionary
    maxnrat = {}
    for rr in data:
        tnam = rr[coln]
        tcond = float(rr[cold])
        if rr[coln] in maxnrat:
            if (decision == 'Highest') and (tnam in maxnrat) and (tcond > maxnrat[tnam]): #use criterion here, only one implemented for now
                maxnrat[tnam] = tcond
        else:
            maxnrat[tnam] = tcond
    
    cdata = [] #cleaned data
    adlist = []
    for rr in data:
        tnam = rr[coln]
        tcond = float(rr[cold])
        if (tnam not in adlist) and (tcond == maxnrat[tnam]):
            adlist.append(rr[coln])
            cdata.append(rr)
            
    return cdata

In [10]:
cfilgoog = clean_data(filgoog,0,3)

### 3.3 Non-english apps
Since the fictional company from this exercise only creates apps directed at english speaking people, the next step of the data cleaning will be to remove apps whose name contains non-english characters. It is possible to do so by using ASCII standards of each character within an app's name. The ASCII of english characters falls between 0 and 127, and can luckily be extracted using a python function. 

Note that emoji's and some symbols' (€) ASCII fall above 127. As such, we need a criteria to allow apps with limited number of characters with ASCII above 127 to be retained in our "english" list. To be able to compare the results obtained here with those of DataQuest tutorial the same criterion (3) is used. However, ideally, more in depth analysis, such as the ratio of non-english to english characters should be used. 

To test whether characters are english ones, the following function provided by the DataQuest tutorial will be used. 

    Note: running the following cell is MANDATORY.

In [11]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

A second function, which I modified form the DataQuest one will be used to clean up the data. 
    
    Note: the function was modified so that it could be applied to both datasets, somethings that was not done by the function suggested in DataQuest's tutorial. 
    Running the following to cells is MANDATORY for the rest of the code to function properly. 

In [12]:
def clean_data_language(data,col_nam):
    #col_nam is the indice of the column containing the name of the app
    tlist = []
    coln = col_nam
    for rr in data:
        if is_english(rr[coln]):
            tlist.append(rr)
    return tlist

In [13]:
#And now to apply the function to the data:
ecfilgoog = clean_data_language(cfilgoog,0)
efilapp = clean_data_language(filapp,2)

### 3.4 Free apps
Lastly, for cleaning at least, the non-free apps must be removed. The reason being that the fictional company only creates free apps. To do so, a modified version of code provided by DataQuest will be used.

    Running the following to cells is MANDATORY for the rest of the code to function properly. Again, the code was improved so that the same function could be used on both datasets. 

In [14]:
def clean_data_free(data,col_cond):
    #col_cond is the column with the price of the app
    tlist = []
    
    for rr in data:
        if rr[col_cond] == '0':
            tlist.append(rr)
        
    return tlist

In [15]:
final_goog = clean_data_free(ecfilgoog,7)
final_app = clean_data_free(efilapp,5)

## Step 4
**Finally done with cleaning the data!**
**In this step, the cleaned up data will be analyzed.**

Now that the data is clean it is possible to analyze it in order to find which kind of app is most likely to be downloaded. As a quick reminder, the fictional company earns money through adds within the apps, the more downloads, the more money they earn. Thus, the goal of this step will be to identify app types that are the most downloaded. We first need to figure out which columns within the datasets can be used to assess the type/category of an app. By taking a look a the description of the fields above, we find that the following ones could be of use: 
 - Google : Category and Genres which correspond to columns 1 and 9 of dataset
 - Apple : prime_genres which correspond to column 12


### 4.1 Looking at categories 
To assess whether each of these fields can be of use , we first need to obtain a list of the possible types that each entry contains. The following function, that I wrote, will allow us to do that. 

    Note: Running the following two cells is MANDATORY for the rest of the code to function properly. 

In [16]:
def frequency_calculator(data,col):
    #This function will create a frequency table of the different types found within a column of the data.
    
    tdict = {} #output variable is a dictionary
    count = 0
    
    for rr in data:
        if isinstance(rr[col],list):
            for ee in rr[col]:
                if ee in tdict:
                    tdict[ee] += 1
                else: 
                    tdict[ee] = 1
                count += 1
        else:
            ee = rr[col]
            if ee in tdict:
                tdict[ee] += 1
            else: 
                tdict[ee] = 1
            count += 1
            
    for kk in tdict:
        tdict[kk] /= count/100
    
    return tdict

In [17]:
freqGoogCat = frequency_calculator(final_goog,1)
freqGoogGen = frequency_calculator(final_goog,9)
freqAppGen = frequency_calculator(final_app,12)

Now let's take a look at the number of categories we find for each as well as the list of these categories.

    Note: this is FACULTATIVE and for visualization purposes only. 

In [18]:
print('Number of different categories for the')
print('Google category: ',len(freqGoogCat),'   Google genre: ',len(freqGoogGen), '   App genre: ', len(freqAppGen), '\n')

for ii in range(0,10):
    print([list(freqGoogCat.keys())[ii],list(freqGoogGen.keys())[ii],list(freqAppGen.keys())[ii]])

Number of different categories for the
Google category:  33    Google genre:  114    App genre:  23 

['ART_AND_DESIGN', 'Art & Design', 'Productivity']
['AUTO_AND_VEHICLES', 'Art & Design;Creativity', 'Weather']
['BEAUTY', 'Auto & Vehicles', 'Shopping']
['BOOKS_AND_REFERENCE', 'Beauty', 'Reference']
['BUSINESS', 'Books & Reference', 'Finance']
['COMICS', 'Business', 'Music']
['COMMUNICATION', 'Comics', 'Utilities']
['DATING', 'Comics;Creativity', 'Travel']
['EDUCATION', 'Communication', 'Social Networking']
['ENTERTAINMENT', 'Dating', 'Sports']


This is kind of hard to read. One way to make a simple and clear table of the frequencies produced by the previous function is to order them so that the most commmon categories appear on top of the list and the least common ones appear at the bottom. DataQuest kindly provides such a function. I have modified it slightly in the cell below. I also added a second function that can take the output of the previous one and display it. The advantage of the second function is that it is made to display multipe dictionaries side by side. 

        Note: Running the first cells is MANDATORY for the rest of the code to function properly. The second cell is for visualization purposes only. 

In [19]:
def display_table(dataset, index, visu = False):
    #Either prints a dictionary ordered by its values (rather than keys) 
    # or outputs the dictionary so that it is 
    table = frequency_calculator(dataset, index)
    table_display = []
    
    ke = list(table.keys())
    val = list(table.values())

    sortke = [x for _,x in sorted(zip(val,ke), reverse = True)]
    sortval =  sorted(val, reverse = True)
    table_sorted = dict(zip(sortke, sortval))

    if visu:
        for entry in table_sorted:
            print("{} {:20}".format(round(table_sorted[entry],1),entry))
    else:
        return table_sorted

def display_mult_tables(tb1,tb2,tb3,col_names=['Google_Categories','Apple_Genre','Google_Genre']):
    
    nlin = max([len(tb1),len(tb2),len(tb3)])
    print("{:25} {:25} {}".format(*col_names))
    
    for ll in range(0,nlin-1):
        try:
            ke1 = list(tb1.keys())[ll]
            va1 = round(tb1[ke1],1)
        except:
            ke1 = ''
            va1 = ''

        try: 
            ke2 = list(tb2.keys())[ll]
            va2 = round(tb2[ke2],1)
        except:
            ke2 = ''
            va2 = ''

        try: 
            ke3 = list(tb3.keys())[ll]
            va3 = round(tb3[ke3],1)
        except:
            ke3 = ''
            va3 = ''
        
        print("{} {:20} {:5} {:20} {:5} {:20}".format(*[va1,ke1,va2,ke2,va3,ke3]))

In [20]:
sortGoogCat = display_table(final_goog,1)
sortGoogGen = display_table(final_goog,9)
sortAppGen = display_table(final_app,12)

display_mult_tables(sortGoogCat,sortAppGen,sortGoogGen)

Google_Categories         Apple_Genre               Google_Genre
18.9 FAMILY                58.2 Games                  8.4 Tools               
9.7 GAME                   7.9 Entertainment          6.1 Entertainment       
8.5 TOOLS                  5.0 Photo & Video          5.3 Education           
4.6 BUSINESS               3.7 Education              4.6 Business            
3.9 LIFESTYLE              3.3 Social Networking      3.9 Productivity        
3.9 PRODUCTIVITY           2.6 Shopping               3.9 Lifestyle           
3.7 FINANCE                2.5 Utilities              3.7 Finance             
3.5 MEDICAL                2.1 Sports                 3.5 Medical             
3.4 SPORTS                 2.0 Music                  3.5 Sports              
3.3 PERSONALIZATION        2.0 Health & Fitness       3.3 Personalization     
3.2 COMMUNICATION          1.7 Productivity           3.2 Communication       
3.1 HEALTH_AND_FITNESS     1.6 Lifestyle              3.1 Action 

Now we have a much clearer idea of what type of free apps can be found in the Apple Store as well as Google play. 

### 4.2 Assessing number of apps downloaded by category
Although we now know the different categories used to describe apps, we still haven't reached our goal of finding what kind of free apps are most downloaded. That is, even though there might be lots of games on apple store, it does not mean that they are more of them downloaded than for utilities. For instance, there might be a single utility app downloaded by everyone while each game is only downloaded by a few people. 

Thus, the next step of the analysis is to create a frequency table of downloaded apps by category. Unfortunately, the apple store dataset does not contain this informaiton. It does contain information about the number of ratings of the app which we will take as a proxy for number of downloads (as suggested by the DataQuest tutorial). This is not ideal, but better than nothing.

Looking at the legend for each of the dataset (see above), it becomes quite clear which fields to use:

 - Google: installs, column 5
 - App: rating_count_tot, column 6
 
 
     Note: Running the following two cells is MANDATORY for the rest of the code to function properly. The third one is for visualization purposes only.

In [21]:
def App_usage(data,col4freq,freqTab,freqTab2data):
    #This function takes a dataset and a dictionary of the categories to be explored. 
    #Two columns must be specified, the one to be searched in the data and the one that 
    #the frequency table was used from
    
    table = {}
    
    for cc in freqTab:
        table[cc] = 0
        count = 0
        
        for rr in data:
            if isinstance(rr[freqTab2data],list):
                for ee in rr[freqTab2data]:
                    if ee == cc:
                        num = rr[col4freq]
                        if num[-1:]== '+':
                            num = num.replace('+','')
                            num = num.replace(',','')
                            table[cc] += int(num)
                            count += 1
                    
                        else:
                            table[cc] += int(num)
                            count += 1
                    
            elif rr[freqTab2data] == cc:
                num = rr[col4freq]
                if num[-1:]== '+':
                    num = num.replace('+','')
                    num = num.replace(',','')
                    table[cc] += int(num)
                    count += 1
                else:
                    table[cc] += int(num)
                    count += 1
                
        table[cc] /= count

    # sort the table
    ke = list(table.keys())
    val = list(table.values())

    sortke = [x for _,x in sorted(zip(val,ke), reverse = True)]
    sortval =  sorted(val, reverse = True)
    
    return  dict(zip(sortke, sortval))


In [22]:
avgUsageGoog = App_usage(final_goog,5,sortGoogCat,1)
avgUsageApp = App_usage(final_app,6,sortAppGen,12)

In [23]:
display_mult_tables(avgUsageGoog,avgUsageApp,{})

Google_Categories         Apple_Genre               Google_Genre
38456119.2 COMMUNICATION        86090.3 Navigation                                     
24727872.5 VIDEO_PLAYERS        74942.1 Reference                                      
23253652.1 SOCIAL               71548.3 Social Networking                              
17840110.4 PHOTOGRAPHY          57326.5 Music                                          
16787331.3 PRODUCTIVITY         52279.9 Weather                                        
15588015.6 GAME                 39758.5 Book                                           
13984077.7 TRAVEL_AND_LOCAL     33333.9 Food & Drink                                   
11640705.9 ENTERTAINMENT        31467.9 Finance                                        
10801391.3 TOOLS                28441.5 Photo & Video                                  
9549178.5 NEWS_AND_MAGAZINES   28243.8 Travel                                         
8767811.9 BOOKS_AND_REFERENCE  26919.7 Shopping         

### Final note
The DataQuest tutorial suggests the analyses that were carried above. However, simply having an average number of app downloaded for each category is quite limited as outliers might be dragging the means. For instance, google map and similarly other well known navigation apps are likely to dominate the field of navigation whereas outside of this small clique, others are likely present in smaller number. To assess such hypothesis in an efficient manner we would need to display histograms. 

